In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split

In [2]:
ecg_feature = pd.read_csv('./ecg_features.csv')
ppg_feature = pd.read_csv('./ppg_features.csv')
eda_feature = pd.read_csv('./eda_features.csv')
feel_feature = pd.read_csv('./feeling_labels.csv')
id_feature = pd.read_csv('./id_labels.csv')

In [3]:
id_feature

,Unnamed: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
1,1,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
2,2,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
3,3,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
4,4,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,229,0.003141,-0.006964,0.010231,0.005548,0.012103,0.000373,0.00070,-0.001923,-0.003356,-0.005007,-0.003531,-0.002360,0.002836,-0.001696,-0.006686,-0.007511,0.015053,0.009562,-0.020179
230,230,0.003141,-0.006964,0.010231,0.005548,0.012103,0.000373,0.00070,-0.001923,-0.003356,-0.005007,-0.003531,-0.002360,0.002836,-0.001696,-0.006686,-0.007511,0.015053,0.009562,-0.020179
231,231,0.003141,-0.006964,0.010231,0.005548,0.012103,0.000373,0.00070,-0.001923,-0.003356,-0.005007,-0.003531,-0.002360,0.002836,-0.001696,-0.006686,-0.007511,0.015053,0.009562,-0.020179
232,232,0.003141,-0.006964,0.010231,0.005548,0.012103,0.000373,0.00070,-0.001923,-0.003356,-0.005007,-0.003531,-0.002360,0.002836,-0.001696,-0.006686,-0.007511,0.015053,0.009562,-0.020179


In [4]:
ecg_feature = ecg_feature.drop(['Unnamed: 0'], axis = 1)
ppg_feature = ppg_feature.drop(['Unnamed: 0'], axis = 1)

In [5]:
X_train = pd.concat([ecg_feature, ppg_feature, eda_feature], axis = 1).values

In [6]:
label_data = pd.read_csv('./train_label_softmax.csv')
y_train = label_data.values

In [7]:
X_train

array([[12.140015, 19.875168,  0.      , ..., 21.984102, 21.86208 ,
        13.78929 ],
       [13.15213 , 21.535746,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [13.004108, 21.283129,  0.      , ..., 24.902727, 23.910677,
        16.990757],
       ...,
       [11.715818, 19.185234,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [12.60052 , 20.63363 ,  0.      , ...,  0.      , 22.506584,
         0.      ],
       [10.416733, 17.057709,  0.      , ..., 30.521257, 30.937357,
         0.      ]])

In [8]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(Flatten())

batch_size = 1
num_samples = 246
features = []

for i in range(0, num_samples, batch_size):
    batch = X_train[i:i+batch_size]
    extracted_features = model.predict(batch)
    features.append(extracted_features)

1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 16ms/step


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [9]:
feat_df = pd.DataFrame(np.array(features).reshape(234, 64))
feat_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.565098,0.0,0.0,0.434361,0.413127,0.395377,1.039579,1.232226,0.0,0.048050,...,0.234508,0.612355,1.040545,1.932615,1.187817,0.0,0.0,0.0,0.282925,0.966323
1,0.576468,0.0,0.0,0.367564,0.358954,0.361457,1.004327,1.126847,0.0,0.049577,...,0.313709,0.678466,1.035463,1.764355,1.145620,0.0,0.0,0.0,0.222110,0.941489
2,0.621588,0.0,0.0,0.463051,0.423417,0.430843,1.132702,1.343189,0.0,0.062867,...,0.254622,0.683359,1.132190,2.120891,1.308342,0.0,0.0,0.0,0.309888,1.061451
3,0.434022,0.0,0.0,0.329771,0.291781,0.285378,0.773319,0.915782,0.0,0.016981,...,0.223369,0.491452,0.803092,1.466831,0.911780,0.0,0.0,0.0,0.203119,0.761408
4,0.488501,0.0,0.0,0.369346,0.325527,0.308550,0.852090,0.989836,0.0,0.021860,...,0.322552,0.560646,0.879981,1.558828,0.989210,0.0,0.0,0.0,0.188919,0.812425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,0.666846,0.0,0.0,0.498998,0.534862,0.473776,1.249997,1.489836,0.0,0.000000,...,0.508878,0.870744,1.310590,2.314929,1.474809,0.0,0.0,0.0,0.316193,1.343392
230,0.627809,0.0,0.0,0.605346,0.490822,0.553056,1.291701,1.366692,0.0,0.000000,...,0.475461,0.903219,1.375106,2.317326,1.469293,0.0,0.0,0.0,0.275546,1.246398
231,0.683780,0.0,0.0,0.600608,0.553493,0.492710,1.412623,1.541578,0.0,0.008269,...,0.593117,0.927975,1.457177,2.356995,1.507502,0.0,0.0,0.0,0.187911,1.331203
232,0.738573,0.0,0.0,0.600964,0.561713,0.534666,1.395498,1.525459,0.0,0.018908,...,0.490707,0.868786,1.536873,2.551521,1.591871,0.0,0.0,0.0,0.231621,1.285934


In [10]:
X_train = pd.concat([feat_df, feel_feature, id_feature], axis = 1).values

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy'])
#model.add(Flatten())
model.fit(X_train, y_train, epochs=500, batch_size=32)

In [ ]:
output_layer = Dense(6, activation='softmax')(x)  

model = Model(inputs=input_layer, outputs=output_layer)

# Test

In [13]:
ecg_test_feature = pd.read_csv('./ecg_test_features.csv')
ppg_test_feature = pd.read_csv('./ppg_test_features.csv')
eda_test_feature = pd.read_csv('./eda_test_features.csv')
#feel_feature = pd.read_csv('./feeling_test_labels.csv')
#id_feature = pd.read_csv('./id_test_labels.csv')

In [14]:
ecg_test_feature = ecg_test_feature.drop(['Unnamed: 0'], axis = 1)
ppg_test_feature = ppg_test_feature.drop(['Unnamed: 0'], axis = 1)

In [15]:
X_test = pd.concat([ecg_test_feature, ppg_test_feature, eda_test_feature], axis = 1).values

In [16]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(Flatten())

batch_size = 1
num_samples = 108
features = []

for i in range(0, num_samples, batch_size):
    batch = X_train[i:i+batch_size]
    extracted_features = model.predict(batch)
    features.append(extracted_features)

1/1 [==============================] - 0s 16ms/step


In [17]:
test_feat_df = pd.DataFrame(np.array(features).reshape(108, 64))
test_feat_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.018825,0.011338,0.0,0.011365,0.0,0.074885,0.0,0.0,0.014245,0.015686,...,0.0,0.0,0.0,0.0,0.068142,0.0,0.051114,0.0,0.0,0.0
1,0.021140,0.010726,0.0,0.012046,0.0,0.076671,0.0,0.0,0.015630,0.016711,...,0.0,0.0,0.0,0.0,0.071021,0.0,0.054741,0.0,0.0,0.0
2,0.031239,0.015493,0.0,0.015754,0.0,0.099335,0.0,0.0,0.022315,0.026235,...,0.0,0.0,0.0,0.0,0.093966,0.0,0.071010,0.0,0.0,0.0
3,0.037438,0.016732,0.0,0.015753,0.0,0.091162,0.0,0.0,0.025739,0.034387,...,0.0,0.0,0.0,0.0,0.091216,0.0,0.067751,0.0,0.0,0.0
4,0.048457,0.021009,0.0,0.021183,0.0,0.111350,0.0,0.0,0.033009,0.045689,...,0.0,0.0,0.0,0.0,0.111693,0.0,0.083444,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.954343,0.447130,0.0,0.422648,0.0,1.668350,0.0,0.0,0.815689,1.068298,...,0.0,0.0,0.0,0.0,1.816956,0.0,1.338437,0.0,0.0,0.0
104,0.960701,0.450393,0.0,0.426557,0.0,1.674518,0.0,0.0,0.823183,1.077552,...,0.0,0.0,0.0,0.0,1.824814,0.0,1.344687,0.0,0.0,0.0
105,0.965111,0.452899,0.0,0.429224,0.0,1.678871,0.0,0.0,0.831656,1.086498,...,0.0,0.0,0.0,0.0,1.830791,0.0,1.349656,0.0,0.0,0.0
106,0.973997,0.457398,0.0,0.433009,0.0,1.691841,0.0,0.0,0.839142,1.096624,...,0.0,0.0,0.0,0.0,1.846311,0.0,1.360774,0.0,0.0,0.0


In [ ]:
test_label_data = pd.read_csv('./test_label_softmax.csv')
y_test = test_label_data.values